In [352]:
import requests
import pandas as pd
import os
import re
import math
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime
from collections import Counter
from natsort import natsorted

In [353]:
episodes = []
title=[]
types = []
popularity = []
description=[]
related=[]
release=[]
animeScore=[]
voices=[]
staff=[]
ranks=[]
# positioning
folder = r"./tsv_anime/"
# iter over the file
for anime in tqdm(natsorted(os.listdir(folder))):
    df = pd.read_csv(folder+anime, sep = "\t")
    episodes.append(df["animeNumEpisode"][0])
    types.append(df["animeType"][0])
    popularity.append(df["animePopularity"][0])
    description.append(df["animeDescription"][0])
    title.append(df["animeTitle"][0])
    related.append(df["animeRelated"][0])
    release.append(df["releaseDate"][0])
    animeScore.append(df["animeScore"][0])
    voices.append(df["animeVoices"][0])
    staff.append(df["animeStaff"][0])
    ranks.append(df["animeRank"][0])
    

100%|█████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:03<00:00, 449.28it/s]


In [354]:
df

,animeTitle,animeType,animeNumEpisode,releaseDate,endDate,animeNumMembers,animeScore,animeUsers,animeRank,animePopularity,animeDescription,animeRelated,animeCharacters,animeVoices,animeStaff
0,Konjiki no Gash Bell!!,TV,150,2003-04-06,2006-03-26,107912,7.54,46265,1499,1519,"Takamine Kiyomaro, a depressed don't-care-abou...","['Konjiki no Gash!!', 'Konjiki no Gash Bell!!:...","['Bell, Gash', 'Takamine, Kiyomaro', 'Folgore,...","['Ootani, IkueJapanese', 'Sakurai, TakahiroJap...","[[['Hamano, Takatoshi', 'Producer'], ['Kaizawa..."


## Change datatypes 

change title[316] datatype to string

In [355]:
import numpy as np
print(type(title[316]))
title[316]=title[316].item()
print(type(title[316]))
title[316]=str(title[316])
print(type(title[316]))

<class 'numpy.int64'>
<class 'int'>
<class 'str'>


In [356]:
for i in range(len(staff)):
    if(type(staff[i])!=str):
        staff[i]=staff[i].item()
        staff[i]=str(staff[i])
    #print(type(staff[i]))

In [357]:
for i in range(len(voices)):
    if(type(voices[i])!=str):
        voices[i]=voices[i].item()
        voices[i]=str(voices[i])
    #print(type(staff[i]))

In [358]:
for i in range(len(ranks)):
    ranks[i]=ranks[i].item()

### We used cosine to determine  how similar the documents are irrespective of their size is to the entered query.Does the text match each other? : (Description,AnimeStaff,Voices)

In [359]:
WORD = re.compile(r"\w+")

In [415]:
def cosine_text(input_text,comparison_text):
    if input_text == "":
        return 0
    intersection = set(input_text.keys()) & set(comparison_text.keys())
    numerator = sum([input_text[x] * comparison_text[x] for x in intersection])
    sum1 = sum([input_text[x] ** 2 for x in list(input_text.keys())])
    sum2 = sum([input_text[x] ** 2 for x in list(comparison_text.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator  

In [406]:
def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [362]:
def cosine(input_text,comparison_text):
    vector1 = text_to_vector(input_text)
    vector2 = text_to_vector(comparison_text)
    cosine = cosine_text(vector1, vector2)
    return(cosine)

### How to text similar:Type

In [407]:
from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

### For num_episode variable, we have classified the amount of anime into 4 categories:
##### 0 is film
##### 1 is short
##### 2 is medium
##### 3 is large


First we define the category of the input data

In [364]:
def identify_category(input_eps,next_eps):
    if(type(next_eps) == str):
        return 0
    score=0
    if(input_eps==1):
        category=0
    elif(input_eps<25 and input_eps>=8):
        category=1
    elif(input_eps<65 and input_eps>=25):
        category=2
    else:
        category=3
        
    if(next_eps==1):
        category1=0
    elif(next_eps<25 and input_eps>=8):
        category1=1
    elif(next_eps<65 and input_eps>=25):
        category1=2
    else:
        category1=3
    return(category,category1)

Calculated their differences. Depending on the number of episodes, we will be able to determine the greatest suitable anime by length

In [388]:
def episode_score(input_eps,next_eps):
    if(type(next_eps)==str):
        return 0
    category=(identify_category(input_eps,next_eps))
    #print(category)
    if(category[0]==category[1]):
        return 1
    elif(abs(category[0]-category[1])==1):
        return 0.66
    elif(abs(category[0]-category[1])>1):
        return 0.33
    else:
        return 0

## Rank

The logic of ranking metric is that we push away from the request, if the query anime was given to us by the top 100 anime, we of course are looking for a suitable one from the top 100 or from the top 1000

In [389]:
def ranking(input_rank,next_rank):
    if input_rank == 0:
        return 0
    diff=abs(input_rank-next_rank)
    if(diff<=100):
        return 1
    elif(diff>100 and diff<=1000):
        return 0.66
    else:
        return 0.33

### Date

We created a date metric to determine how late and newer the anime is with the query being compared.

In [390]:
def date(input_date,releaseDate):
    diff = (abs(datetime.strptime(input_date,'%Y-%m-%d')-datetime.strptime(releaseDate,'%Y-%m-%d')))
    days = diff.days
    #Anime came out in the same season
    if(days>0 and days<=100):
        return 1
    #Differences in 1 year
    elif(days>100 and days<=300):
        return 0.5
    return(0)

print(date(date_query,release[0]))

0


## Popularity metric

The logic of this metric is similar to the rank metric.

In [391]:
def score_pop(input_pop,next_pop):
    if input_pop == 0:
        return 0
    diff=abs(input_pop-next_pop)
    if(input_pop<=100 and diff<=100):
        return 0.66
    elif(input_pop > 100 and input_pop <= 1000 and diff > 100):
        return 0.44
    else:
        return 0.22

## animeScore metric

We also categorized the anime by a rating, like metric number episodes
#### 1 High  
#### 2 Average 
#### 3 Low

In [402]:
def identify_s(input_s,next_s):
    if input_s == 0:
        return 0
    if(input_s>=8):
        category=1
    elif(input_s<8 and input_s>=5):
        category=2
    else:
        category=3
       
    if(next_s>=8):
        category1=1
    elif(next_s<8 and next_s>=5):
        category1=2
    else:
        category1=3
    return(category,category1)

In [393]:
def score_s(input_s,next_s):
    if input_s == 0:
        return 0
    category = identify_s(input_s,next_s)
    #print(category)
    if(category[0]==category[1]):
        return 1
    elif(abs(category[0]-category[1])==1):
        return 0.66
    elif(abs(category[0]-category[1])>1):
        return 0.33
    else:
        return 0

In [316]:
#for i in range(len(popularity)):
    #print(popularity[i])

### Final score metric

In [429]:
def final_score1(i,input_type,next_type,input_eps,next_eps,input_title,next_title,input_related,next_related,input_description,next_description,input_date,next_date,input_voice,next_voice,input_staff,next_staff):
    #Type variable
    print(title[i])
    score1=cosine(input_type,next_type)
    print("Type_score",score1)
    #Episodes variable
    score2=episode_score(input_eps,next_eps)
    print("Episode_score",score2)
    #Title variable
    score3=cosine(input_title,next_title)
    print("Title_score",score3)
    #Related anime variable
    score4=similar(input_related,next_related)
    print("Related_score",score4)
    #Description variable
    score5=cosine(input_description,next_description)
    print("Description_score",score5)
    #Season date variable
    score6=date(input_date,next_date)
    print("Season_score",score6)
    #Voices variable
    score7=cosine(input_voice,next_voice)
    print("Voice",score7)
    #Staff  variable
    score8=cosine(input_staff,next_staff)
    print("Staff",score8)
    return(score1*0.5+score2*0.5+score3*2+score4+score5*2+score6*0.5+score7*0.3+score8*0.3)

In [425]:
def final_score2(input_rank,next_rank,input_pop,next_pop,input_s,next_s):
    #Rank
    score9 = ranking(input_rank,next_rank)
    print("Rank",score9)
    #Popularity
    score10=score_pop(input_pop,next_pop)
    print("Popular",score10)
    #animeScore
    score11=score_s(input_s,next_s)
    print("animeScore",score11)
    return(score9+score10+score11)


In [373]:
with open("./anime_url.txt", "r", encoding = "utf-8") as f:
    urls = f.readlines()

In [430]:
title_query="Gintama"
type_query=""
ep_query=54
related_query=""
description_query="Gintoki, Shinpachi, and Kagura return as the fun-loving Edo Sword Japan s are prohibited and alien overlords have conquered Japan, they try to thrive on doing whatever work they can get their hands on. However, Shinpachi and Kagura still haven't been paid... Does Gin-chan really spend all that cash playing pachinko?"

date_query="2015-06-21"
voices_query=""
staff_query=""

rank_query=2
pop_query=337
score_query=9.09
scores=[]

for i in range(len(episodes)):
    score1=final_score1(i,type_query,types[i],ep_query,episodes[i],title_query,title[i],related_query,related[i],description_query,description[i],date_query,release[0],voices_query,voices[i],staff_query,staff[i])
    score2=final_score2(rank_query,ranks[i],pop_query,popularity[i],score_query,animeScore[i])
    score=score1+score2
    scores.append([score,i])
sorted_scores=sorted(scores, reverse=True)


df_score=[]
df_title=[]
df_urls=[]
for i in range(len(episodes)):
    df_score.append(sorted_scores[i][0])
    df_title.append(title[sorted_scores[i][1]])
    df_urls.append(urls[sorted_scores[i][1]])
df1 = pd.DataFrame (df_score, columns = ['Score:'])
df2 = pd.DataFrame (df_title, columns = ['Title:'])
df3 = pd.DataFrame (df_urls, columns = ['URL:'])
pdList = [df1, df2, df3 ]  
new_df = pd.concat(pdList, axis=1)
display(new_df)

Fullmetal Alchemist: Brotherhood
Type_score 0.0
Episode_score 1
Title_score 0.0
Related_score 0.0
Description_score 1.1466825857145266
Season_score 0
Voice 0.0
Staff 0.0
Rank 1
Popular 0.44
animeScore 1
Gintama°
Type_score 0.0
Episode_score 1
Title_score 1.0
Related_score 0.0
Description_score 1.6259413581697955
Season_score 0
Voice 0.0
Staff 0.0
Rank 1
Popular 0.22
animeScore 1
Shingeki no Kyojin Season 3 Part 2
Type_score 0.0
Episode_score 0.66
Title_score 0.0
Related_score 0.0
Description_score 0.8409178658720821
Season_score 0
Voice 0.0
Staff 0.0
Rank 1
Popular 0.44
animeScore 1
Steins;Gate
Type_score 0.0
Episode_score 0.66
Title_score 0.0
Related_score 0.0
Description_score 1.0145993123917847
Season_score 0
Voice 0.0
Staff 0.0
Rank 1
Popular 0.44
animeScore 1
Fruits Basket: The Final
Type_score 0.0
Episode_score 0.66
Title_score 0.0
Related_score 0.0
Description_score 1.1653478941010338
Season_score 0
Voice 0.0
Staff 0.0
Rank 1
Popular 0.44
animeScore 1
Gintama'
Type_score 0.0
Epi

Related_score 0.0
Description_score 1.1215271415581973
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.66
Popular 0.22
animeScore 1
Haikyuu!!
Type_score 0.0
Episode_score 1
Title_score 0.0
Related_score 0.0
Description_score 1.0692676621563626
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.66
Popular 0.44
animeScore 1
Kimetsu no Yaiba: Mugen Ressha-hen
Type_score 0.0
Episode_score 0.66
Title_score 0.0
Related_score 0.0
Description_score 0.2988071523335984
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.66
Popular 0.44
animeScore 1
Kobayashi-san Chi no Maid Dragon S
Type_score 0.0
Episode_score 0.66
Title_score 0.0
Related_score 0.0
Description_score 0.8918825850158446
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.66
Popular 0.44
animeScore 1
Holo no Graffiti
Type_score 0.0
Episode_score 0
Title_score 0.0
Related_score 0.0
Description_score 0.4472135954999579
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.66
Popular 0.44
animeScore 1
Mahou Shoujo Madoka★Magica Movie 3: Hangyaku no Monogatari
Type_score 0.0
Episo

Rank 0.66
Popular 0.44
animeScore 1
Redline
Type_score 0.0
Episode_score 0.33
Title_score 0.0
Related_score 0.0
Description_score 1.1713452477561832
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.66
Popular 0.44
animeScore 1
Stranger: Mukou Hadan
Type_score 0.0
Episode_score 0.33
Title_score 0.0
Related_score 0.0
Description_score 0.9759000729485332
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.66
Popular 0.44
animeScore 1
Tokyo Revengers
Type_score 0.0
Episode_score 0.66
Title_score 0.0
Related_score 0.0
Description_score 1.246111965698067
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.66
Popular 0.44
animeScore 1
Ashita no Joe
Type_score 0.0
Episode_score 0.66
Title_score 0.0
Related_score 0.0
Description_score 0.9871352947927471
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.66
Popular 0.44
animeScore 1
Bakemono no Ko
Type_score 0.0
Episode_score 0.33
Title_score 0.0
Related_score 0.0
Description_score 1.486675513228308
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.66
Popular 0.44
animeScore 1
Initial D F

Season_score 0
Voice 0.0
Staff 0.0
Rank 0.66
Popular 0.44
animeScore 1
Ouran Koukou Host Club
Type_score 0.0
Episode_score 1
Title_score 0.0
Related_score 0.0
Description_score 1.121223821162776
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.66
Popular 0.44
animeScore 1
Tanoshii Muumin Ikka
Type_score 0.0
Episode_score 0.66
Title_score 0.0
Related_score 0.0
Description_score 1.1492233402234249
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.66
Popular 0.44
animeScore 1
Working!!!: Lord of the Takanashi
Type_score 0.0
Episode_score 0.33
Title_score 0.0
Related_score 0.0
Description_score 0.7027642214999339
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.66
Popular 0.44
animeScore 1
Detective Conan Movie 20: The Darkest Nightmare
Type_score 0.0
Episode_score 0.33
Title_score 0.0
Related_score 0.0
Description_score 1.6494154808814632
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.66
Popular 0.44
animeScore 1
Akira
Type_score 0.0
Episode_score 0.33
Title_score 0.0
Related_score 0.0
Description_score 1.041976144

Type_score 0.0
Episode_score 0.66
Title_score 0.0
Related_score 0.0
Description_score 0.598056818096707
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.66
Popular 0.44
animeScore 1
Love Live! Superstar!!
Type_score 0.0
Episode_score 0.66
Title_score 0.0
Related_score 0.0
Description_score 1.299600338165945
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.66
Popular 0.44
animeScore 1
Black Clover
Type_score 0.0
Episode_score 0.66
Title_score 0.0
Related_score 0.0
Description_score 1.4394526075990866
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.66
Popular 0.44
animeScore 1
City Hunter 2
Type_score 0.0
Episode_score 1
Title_score 0.0
Related_score 0.0
Description_score 0.5217939320872036
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.66
Popular 0.44
animeScore 1
Girls & Panzer: Saishuushou Part 2
Type_score 0.0
Episode_score 0.33
Title_score 0.0
Related_score 0.0
Description_score 1.1516578439248717
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.66
Popular 0.44
animeScore 1
Kamisama Hajimemashita
Type_score 0.0
Ep

...

Description_score 1.101564967739652
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.33
Popular 0.44
animeScore 0.66
Yuru Yuri,
Type_score 0.0
Episode_score 0.33
Title_score 0.0
Related_score 0.0
Description_score 0.8017837257372731
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.33
Popular 0.44
animeScore 0.66
Arslan Senki (TV)
Type_score 0.0
Episode_score 1
Title_score 0.0
Related_score 0.0
Description_score 1.5501152030920613
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.33
Popular 0.44
animeScore 0.66
Bishoujo Senshi Sailor Moon
Type_score 0.0
Episode_score 1
Title_score 0.0
Related_score 0.0
Description_score 0.9860132971832695
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.33
Popular 0.44
animeScore 0.66
Bishoujo Senshi Sailor Moon Eternal Movie 2
Type_score 0.0
Episode_score 0.33
Title_score 0.0
Related_score 0.0
Description_score 1.3801311186847085
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.33
Popular 0.44
animeScore 0.66
Bokura wa Minna Kawai-sou
Type_score 0.0
Episode_score 0.66
Title_score 0.0
Rel

Description_score 0.630669771673144
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.33
Popular 0.44
animeScore 0.66
Luo Xiao Hei Zhan Ji
Type_score 0.0
Episode_score 1
Title_score 0.0
Related_score 0.0
Description_score 0.27105237087157535
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.33
Popular 0.44
animeScore 0.66
Lupin III
Type_score 0.0
Episode_score 0.66
Title_score 0.0
Related_score 0.0
Description_score 0.7129310462392062
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.33
Popular 0.44
animeScore 0.66
Lupin III: Part 6
Type_score 0.0
Episode_score 0.66
Title_score 0.0
Related_score 0.0
Description_score 0.5976143046671968
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.33
Popular 0.44
animeScore 0.66
Machikado Mazoku
Type_score 0.0
Episode_score 0.66
Title_score 0.0
Related_score 0.0
Description_score 1.261339543346288
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.33
Popular 0.44
animeScore 0.66
Maria-sama ga Miteru: Haru
Type_score 0.0
Episode_score 0.66
Title_score 0.0
Related_score 0.0
Description_scor

Rank 0.33
Popular 0.44
animeScore 0.66
Senjou no Valkyria
Type_score 0.0
Episode_score 1
Title_score 0.0
Related_score 0.0
Description_score 0.9683296637314884
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.33
Popular 0.44
animeScore 0.66
Senki Zesshou Symphogear AXZ
Type_score 0.0
Episode_score 0.66
Title_score 0.0
Related_score 0.0
Description_score 1.5537971921347118
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.33
Popular 0.44
animeScore 0.66
The iDOLM@STER Movie: Kagayaki no Mukougawa e!
Type_score 0.0
Episode_score 0.33
Title_score 0.0
Related_score 0.0
Description_score 0.7725894187328016
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.33
Popular 0.44
animeScore 0.66
Uchuu Kyoudai: Number Zero
Type_score 0.0
Episode_score 0.33
Title_score 0.0
Related_score 0.0
Description_score 0.3380617018914066
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.33
Popular 0.44
animeScore 0.66
Utawarerumono Specials
Type_score 0.0
Episode_score 0.66
Title_score 0.0
Related_score 0.0
Description_score 0.276026223736941

Episode_score 0.33
Title_score 0.0
Related_score 0.0
Description_score 0.11952286093343936
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.33
Popular 0.44
animeScore 0.66
Yama no Susume: Third Season
Type_score 0.0
Episode_score 0.66
Title_score 0.0
Related_score 0.0
Description_score 0.747017880833996
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.33
Popular 0.44
animeScore 0.66
Yaoguai Mingdan 2nd Season
Type_score 0.0
Episode_score 0.66
Title_score 0.0
Related_score 0.0
Description_score 0.1690308509457033
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.33
Popular 0.44
animeScore 0.66
Yondemasu yo, Azazel-san. Z
Type_score 0.0
Episode_score 0.66
Title_score 0.0
Related_score 0.0
Description_score 0.0
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.33
Popular 0.44
animeScore 0.66
Planetarian: Chiisana Hoshi no Yume
Type_score 0.0
Episode_score 0.66
Title_score 0.0
Related_score 0.0
Description_score 0.7951923208052685
Season_score 0
Voice 0.0
Staff 0.0
Rank 0.33
Popular 0.44
animeScore 0.66
Saki Achiga-hen

,Score:,Title:,URL:
0,8.581411,Gintama: Yorinuki Gintama-san on Theater 2D,https://myanimelist.net/anime/21899/Gintama__Y...
1,7.971883,Gintama°,https://myanimelist.net/anime/28977/Gintama°\n
2,7.632876,Gintama',https://myanimelist.net/anime/9969/Gintama\n
3,7.442201,Gintama.,https://myanimelist.net/anime/34096/Gintama\n
4,7.396397,Gintama.: Shirogane no Tamashii-hen,https://myanimelist.net/anime/36838/Gintama__S...
...,...,...,...
1495,1.595000,Overlord Movie 1: Fushisha no Ou,https://myanimelist.net/anime/34161/Overlord_M...
1496,1.595000,Tamayura: Sotsugyou Shashin Part 1 - Kizashi,https://myanimelist.net/anime/25729/Tamayura__...
1497,1.595000,Overlord Movie 2: Shikkoku no Eiyuu,https://myanimelist.net/anime/34428/Overlord_M...
1498,1.595000,Detective Conan Movie 24: The Scarlet Bullet,https://myanimelist.net/anime/39764/Detective_...
